In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import os
import numpy as np

In [518]:
pd.set_option('display.max_rows', 100)

In [3]:
def voice_usa_crawl(season):
    url = "https://en.wikipedia.org/wiki/The_Voice_(American_season_{})".format(season)
    html = requests.get(url).content
    soup = BeautifulSoup(html, "lxml")
    input_1 = []
    input_2 = []
    pre_extraction = []
    
    for i in range(sum(map(lambda x: "Episode" in x, [row.text for row in soup.find_all('h3')]))+2):
        input_level_1 = soup.find_all('table',{'class':'wikitable'})[i]
        input_level_2 = input_level_1.find_all('tr')
        input_level_3 = [row.text.replace("\n\n", ",").replace("\n", ",").replace("\xa0", "").lstrip(",").split(",") for row in input_level_2]
        pre_extraction.append(input_level_3)
    extraction = [item for episode in pre_extraction for item in episode]
    return(extraction)
    

In [4]:
d = {}
for season in range(1,18):
    d[season] = pd.DataFrame(voice_usa_crawl(season))

In [5]:
desired_order = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,17,16]

In [6]:
d_d = {k: d[k] for k in desired_order}

In [7]:
all_seasons = pd.DataFrame()
for key, value in d_d.items():
     df = value
     df.loc[:,'Season'] = key
     all_seasons = pd.concat([df, all_seasons], axis = 0, sort=True)
all_seasons = all_seasons.iloc[:,:-2]

In [ ]:
all_seasons.head(25)

In [8]:
all_seasons.Season.nunique()

17

In [35]:
colnames = ["Wiki_id", "Singer", "Age", "City", 'State', 'Song', 'Coach_1', 'Coach_2', 'Coach_3', 'Coach_4',
            'extra_1', 'extra_2', 'extra_3', 'Season']

In [36]:
all_seasons.columns = colnames

In [37]:
all_seasons.head(2)

,Wiki_id,Singer,Age,City,State,Song,Coach_1,Coach_2,Coach_3,Coach_4,extra_1,extra_2,extra_3,Season
0,Coaches,Top 54 artists,,None,None,None,None,None,None,None,None,None,None,16
1,Adam Levine,,,None,None,None,None,None,None,None,None,None,None,16


In [13]:
quasi_table_1 = all_seasons.loc[all_seasons.Wiki_id.str.isdigit()]

In [14]:
len(quasi_table_1)

1167

In [15]:
quasi_table_2 = quasi_table_1.loc[((quasi_table_1.Coach_1 == '✔') | (quasi_table_1.Coach_1 == '—')
            | (quasi_table_1.Coach_1 == '✘') | (quasi_table_1.Coach_1 == 'Team full') 
            | (quasi_table_1.Coach_1 == '–') | (quasi_table_1.Coach_1.str.contains('"', na=False)) 
            | (quasi_table_1.Coach_2.str.contains('"', na=False))) & (quasi_table_1.Wiki_id.str.isdigit())]

In [16]:
len(quasi_table_2)

1136

In [17]:
quasi_table_3 = quasi_table_2.loc[~quasi_table_2.Age.str.contains('"', na=False)]

In [18]:
len(quasi_table_3)

1127

In [311]:
quasi_table_3.head()

,Wiki_id,Singer,Age,City,State,Song,Coach_1,Coach_2,Coach_3,Coach_4,extra_1,extra_2,extra_3,Season
24,1,Gyth Rigdon,24,Singer,Louisiana,"""Drift Away""",—,✘,✔,✔,,None,None,16
25,2,Maelyn Jarmon,26,Frisco,Texas,"""Fields of Gold""",✔,✔,✔,✔,,None,None,16
26,3,Karen Galera,19,Dallas,Texas,"""Mi Corazoncito""",—,✔,✔,—,,None,None,16
27,4,Trey Rose,27,Hugo,Oklahoma,"""Wake Me Up""",✔,—,—,✔,,None,None,16
28,5,Kim Cherry,30,Niceville,Florida,"""No Scrubs""",—,—,✔,✔,,None,None,16


In [313]:
quasi_table_3 = quasi_table_3.reset_index(drop=True)

In [314]:
quasi_table_3.head()

,Wiki_id,Singer,Age,City,State,Song,Coach_1,Coach_2,Coach_3,Coach_4,extra_1,extra_2,extra_3,Season
0,1,Gyth Rigdon,24,Singer,Louisiana,"""Drift Away""",—,✘,✔,✔,,None,None,16
1,2,Maelyn Jarmon,26,Frisco,Texas,"""Fields of Gold""",✔,✔,✔,✔,,None,None,16
2,3,Karen Galera,19,Dallas,Texas,"""Mi Corazoncito""",—,✔,✔,—,,None,None,16
3,4,Trey Rose,27,Hugo,Oklahoma,"""Wake Me Up""",✔,—,—,✔,,None,None,16
4,5,Kim Cherry,30,Niceville,Florida,"""No Scrubs""",—,—,✔,✔,,None,None,16


In [316]:
st1 = quasi_table_3.loc[~quasi_table_3.Coach_1.str.contains('"')]
st1 = st1.loc[~st1.Coach_2.str.contains('"')]
st1 = st1.loc[~st1.Song.str.contains('—')]

In [358]:
st2 = quasi_table_3.loc[(quasi_table_3.Coach_1.str.contains('"'))|(quasi_table_3.Coach_2.str.contains('"'))]

In [318]:
st3 = quasi_table_3.loc[(quasi_table_3.Song == '—')]

In [319]:
len(st1)

1066

In [359]:
len(st2)

52

In [321]:
len(st3)

9

In [322]:
st3

,Wiki_id,Singer,Age,City,State,Song,Coach_1,Coach_2,Coach_3,Coach_4,extra_1,extra_2,extra_3,Season
815,6,Yasmin,N/A,N/A,"""Love Song""",—,—,—,—,,None,None,None,5
940,4,Jasmin Rose,N/A,N/A,"""Mercy""",—,—,—,—,,None,None,None,3
991,17,LaRae Rhodes,N/A,N/A,"""Black Horse and the Cherry Tree""",—,—,—,—,,None,None,None,3
1009,10,Gedina,N/A,N/A,"""Ain't Nobody""",—,—,,None,None,None,None,None,3
1027,5,Aly Jados,N/A,N/A,"""Because the Night""",—,—,—,—,,None,NaN,NaN,2
1051,14,Moses Stone,26,Maryland,"""Let's Get It Started""",—,—,✔,—,,None,NaN,NaN,2
1064,12,Luna Searles,N/A,N/A,"""Come to My Window""",—,—,—,—,,None,NaN,NaN,2
1085,12,Beta,N/A,N/A,"""You Make Me Feel...""",—,Team full,,None,None,None,NaN,NaN,2
1116,13,Serabee,34,Mississippi,"""Son of a Preacher Man""",—,—,—,✔,,None,None,NaN,1


In [323]:
st3 = st3.copy()

In [324]:
st3 = st3.loc[:,['Singer','Age','City','State','Season']]

In [325]:
st3.head()

,Singer,Age,City,State,Season
815,Yasmin,N/A,N/A,"""Love Song""",5
940,Jasmin Rose,N/A,N/A,"""Mercy""",3
991,LaRae Rhodes,N/A,N/A,"""Black Horse and the Cherry Tree""",3
1009,Gedina,N/A,N/A,"""Ain't Nobody""",3
1027,Aly Jados,N/A,N/A,"""Because the Night""",2


In [326]:
the_columns = ['Singer', 'Age', 'State', 'Song', 'Season']

In [327]:
st3.columns = the_columns

In [328]:
st3.head()

,Singer,Age,State,Song,Season
815,Yasmin,N/A,N/A,"""Love Song""",5
940,Jasmin Rose,N/A,N/A,"""Mercy""",3
991,LaRae Rhodes,N/A,N/A,"""Black Horse and the Cherry Tree""",3
1009,Gedina,N/A,N/A,"""Ain't Nobody""",3
1027,Aly Jados,N/A,N/A,"""Because the Night""",2


In [329]:
the_columns_ordered = ['Season', 'Singer', 'Age', 'Song', 'State']

In [330]:
st3 = st3[the_columns_ordered]

In [331]:
st3

,Season,Singer,Age,Song,State
815,5,Yasmin,N/A,"""Love Song""",N/A
940,3,Jasmin Rose,N/A,"""Mercy""",N/A
991,3,LaRae Rhodes,N/A,"""Black Horse and the Cherry Tree""",N/A
1009,3,Gedina,N/A,"""Ain't Nobody""",N/A
1027,2,Aly Jados,N/A,"""Because the Night""",N/A
1051,2,Moses Stone,26,"""Let's Get It Started""",Maryland
1064,2,Luna Searles,N/A,"""Come to My Window""",N/A
1085,2,Beta,N/A,"""You Make Me Feel...""",N/A
1116,1,Serabee,34,"""Son of a Preacher Man""",Mississippi


In [332]:
st2

,Wiki_id,Singer,Age,City,State,Song,Coach_1,Coach_2,Coach_3,Coach_4,extra_1,extra_2,extra_3,Season
14,3,The Bundys(Megan,Katey,and Ryan),25-31,Cincinnati,Ohio,"""Closer to Fine""",—,—,✔,✔,,16
38,4,Denton Arnell,32,Chicago,Illinois,"""Hold On","We're Going Home""",—,✔,—,—,,None,16
71,11,Marina Chello,37,Samarkand,Uzbekistan / Plainview,New York,"""Walk Me Home""",✔,—,—,✔,,None,17
159,3,Hannah Blaylock,31,Nimmons,Arkansas,"""Baby","Now That I've Found You""",—,—,—,—,,None,15
175,7,Funsho,29,Lagos,Nigeria / Los Angeles,California,"""Finesse""",✔,✔,✔,—,,None,15
204,4,Mitch Cardoza,22,Dartmouth,Massachusetts,"""No Woman","No Cry""",—,—,—,—,,None,14
209,9,Jorge Eduardo,19,Guadalajara,Mexico / Dallas,Texas,"""Despacito""",—,—,✔,—,,None,14
211,11,Angel Bonilla,31,Quezon City,Philippines / New York City,New York,"""Lay Me Down""",✔,—,—,—,,None,14
246,4,Xaris,17,Gulf Breeze,Florida,"""Don't Think Twice","It's All Right""",—,—,—,—,,None,13
266,7,Kathrina Feigh,24,Quezon City,Philippines / New York City,New York,"""Big White Room""",—,—,✔,✔,,None,13


In [333]:
usa = pd.read_excel("usa_states.ods", engine="odf")

In [334]:
usa.head()

,State,Code
0,D.C.,DC
1,Alabama,AL
2,Alaska,AK
3,Arizona,AZ
4,Arkansas,AR


In [335]:
states_list = usa.State.to_list()

In [360]:
st2_2 = st2.iloc[1:,:]

In [361]:
st2_2.head()

,Wiki_id,Singer,Age,City,State,Song,Coach_1,Coach_2,Coach_3,Coach_4,extra_1,extra_2,extra_3,Season
38,4,Denton Arnell,32,Chicago,Illinois,"""Hold On","We're Going Home""",—,✔,—,—,,None,16
71,11,Marina Chello,37,Samarkand,Uzbekistan / Plainview,New York,"""Walk Me Home""",✔,—,—,✔,,None,17
159,3,Hannah Blaylock,31,Nimmons,Arkansas,"""Baby","Now That I've Found You""",—,—,—,—,,None,15
175,7,Funsho,29,Lagos,Nigeria / Los Angeles,California,"""Finesse""",✔,✔,✔,—,,None,15
204,4,Mitch Cardoza,22,Dartmouth,Massachusetts,"""No Woman","No Cry""",—,—,—,—,,None,14


In [362]:
st2_2["Song_ok"] = np.where((st2_2["Song"].str.contains('"')) & (st2_2["Coach_1"].str.contains('"')),
                               st2_2['Song'].str.cat(st2_2['Coach_1'],sep=", "), 
                               np.where((st2_2["Coach_1"].str.contains('"')) & (st2_2["Coach_2"].str.contains('"')),
                                        st2_2['Coach_1'].str.cat(st2_2['Coach_2'],sep=", "),
                                        np.where(st2_2["Coach_1"].str.contains('"'), st2_2["Coach_1"],
                                                 st2_2["Coach_2"])))

/home/salva/.local/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [363]:
st2_2["State_ok"] = np.where(st2_2["Song"].str.contains('/'), st2_2["Coach_1"], 
                               np.where(st2_2["State"].str.contains('/'), 
                                        np.where(st2_2["Song"].str.strip().isin(states_list), st2_2["Song"], 
                                        st2_2['State'].str.split('/').str[0]), 
                                            np.where(st2_2["State"].str.strip().isin(states_list), st2_2["State"], 
                                            st2_2["Song"])))

/home/salva/.local/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [364]:
st2_2 = st2_2.iloc[:, np.r_[1:3, -3:0]]

In [365]:
st2_2.head(2)

,Singer,Age,Season,Song_ok,State_ok
38,Denton Arnell,32,16,"""Hold On, We're Going Home""",Illinois
71,Marina Chello,37,17,"""Walk Me Home""",New York


In [366]:
st2_2_colnames = ["Singer", "Age", "Season", "Song", "State"]

In [367]:
st2_2.columns = st2_2_colnames

In [368]:
st2_2 = st2_2[the_columns_ordered]

In [369]:
st2_2.head()

,Season,Singer,Age,Song,State
38,16,Denton Arnell,32,"""Hold On, We're Going Home""",Illinois
71,17,Marina Chello,37,"""Walk Me Home""",New York
159,15,Hannah Blaylock,31,"""Baby, Now That I've Found You""",Arkansas
175,15,Funsho,29,"""Finesse""",California
204,14,Mitch Cardoza,22,"""No Woman, No Cry""",Massachusetts


In [370]:
st2_1 = st2.copy()

In [371]:
st2_1["Singer"] = np.where(st2_1["Singer"].str.contains("\("),
                    st2_1["Singer"].str.split("(").str[0], st2_1["Singer"])

In [372]:
st2_1 = st2_1.iloc[:, np.r_[1, 4, 6:8, -1]]

In [373]:
st2_1.iloc[0,1] = 28

In [374]:
st2_1.columns = ["Singer", "Age", "State", "Song", "Season"]

In [375]:
st2_1 = st2_1[the_columns_ordered]

In [376]:
st2_1 = st2_1.iloc[0:1,:]

In [377]:
st2_1

,Season,Singer,Age,Song,State
14,16,The Bundys,28,"""Closer to Fine""",Ohio


In [378]:
st2_3 = pd.concat([st2_2, st2_1], axis=0, sort=False)

In [379]:
st23 = pd.concat([st3, st2_3], axis=0, sort=False)

In [380]:
st23

,Season,Singer,Age,Song,State
815,5,Yasmin,N/A,"""Love Song""",N/A
940,3,Jasmin Rose,N/A,"""Mercy""",N/A
991,3,LaRae Rhodes,N/A,"""Black Horse and the Cherry Tree""",N/A
1009,3,Gedina,N/A,"""Ain't Nobody""",N/A
1027,2,Aly Jados,N/A,"""Because the Night""",N/A
...,...,...,...,...,...
1008,3,Jay Taylor,N/A,"""Country Girl (Shake It for Me)""",Tennessee
1010,3,Taylor Wagner,N/A,"""Mean""",Tennessee
1086,2,Aaron Gordon,N/A,"""So Sick""",Canada
1114,1,Tyler Robinson,20,"""Hey, Soul Sister""",California


In [382]:
st1.head()

,Wiki_id,Singer,Age,City,State,Song,Coach_1,Coach_2,Coach_3,Coach_4,extra_1,extra_2,extra_3,Season
0,1,Gyth Rigdon,24,Singer,Louisiana,"""Drift Away""",—,✘,✔,✔,,None,None,16
1,2,Maelyn Jarmon,26,Frisco,Texas,"""Fields of Gold""",✔,✔,✔,✔,,None,None,16
2,3,Karen Galera,19,Dallas,Texas,"""Mi Corazoncito""",—,✔,✔,—,,None,None,16
3,4,Trey Rose,27,Hugo,Oklahoma,"""Wake Me Up""",✔,—,—,✔,,None,None,16
4,5,Kim Cherry,30,Niceville,Florida,"""No Scrubs""",—,—,✔,✔,,None,None,16


In [383]:
st1 = st1.iloc[:,np.r_[1:3,4:6,13]]

In [384]:
st1 = st1[the_columns_ordered]

In [584]:
st123 = pd.concat([st23, st1], axis=0, sort=False)

In [585]:
st123.sort_index(inplace=True)

In [586]:
st123.Age.value_counts()

25         67
17         65
23         65
26         64
20         64
16         64
24         63
N/A        60
21         55
19         53
22         49
28         46
18         46
29         43
27         40
15         35
31         34
30         33
33         27
35         22
34         20
32         18
37          9
38          8
14          7
40          6
36          6
45          4
52          4
39          4
42          4
13          4
41          3
44          3
54          2
56          2
46          2
51          2
50          1
17 & 19     1
35 & 28     1
32 & 26     1
23 & 44     1
13-14       1
25/23       1
44 / 36     1
55          1
66          1
28          1
27-28       1
43          1
64          1
61          1
62          1
19 & 20     1
13[a]       1
30 / 32     1
24 & 27     1
49          1
48          1
19 / 18     1
35–36       1
Name: Age, dtype: int64

In [587]:
st123["Age"] = st123.Age.apply(str)

In [588]:
st123["Age"] = st123["Age"].apply(lambda x: x.replace("N/A", "na")) 

In [589]:
st123["Age"] = st123["Age"].apply(lambda x: x.replace("&", "-").replace("/", "-").replace("–", "-")
                                                 .replace("[", "-"))

In [590]:
st123["Age"] = np.where(st123["Age"].str.contains('-'),
                        st123['Age'].str.split('-').str[0], st123['Age']) 

In [591]:
st123["Age"] = st123["Age"].apply(lambda x: x.replace("na", "N/A")) 

In [592]:
st123.Age.value_counts()

25     68
17     65
23     65
26     64
20     64
16     64
24     63
N/A    60
21     55
19     53
22     49
28     47
18     46
29     43
27     41
15     35
31     34
30     33
33     27
35     23
34     20
32     18
37      9
38      8
14      7
36      6
13      6
40      6
45      4
52      4
39      4
42      4
41      3
44      3
19      2
56      2
54      2
46      2
51      2
44      1
17      1
23      1
66      1
24      1
64      1
62      1
30      1
50      1
49      1
48      1
55      1
43      1
35      1
61      1
32      1
Name: Age, dtype: int64

In [593]:
st123.State.value_counts()

 California              165
 Texas                   109
 New York                 88
 Florida                  81
 Tennessee                70
 Georgia                  57
 Illinois                 44
 New Jersey               39
 Pennsylvania             31
 Louisiana                27
 Massachusetts            26
 North Carolina           25
 Alabama                  22
 Washington               21
 Virginia                 19
 Ohio                     18
 Missouri                 18
 Michigan                 18
 Kentucky                 18
 Oklahoma                 17
 Utah                     15
 Arkansas                 14
 South Carolina           13
 Minnesota                12
 Indiana                  12
 Connecticut              11
 Maryland                 11
 Colorado                  8
 Nevada                    7
 Iowa                      7
 Arizona                   7
N/A                        7
 Rhode Island              7
 Mississippi               6
 D.C.         

In [594]:
st123.Season = st123.Season.apply(str)

In [595]:
st123 = st123.applymap(lambda x: x.strip())

In [596]:
st123.at[st123.loc[st123.State == 'IL',:].index[0], "State"] = 'Illinois'

In [597]:
st123.at[st123.loc[st123.State == 'Illiinois/Indiana',:].index[0], "State"] = 'Illinois'

In [598]:
st123.at[st123.loc[st123.State == 'Georgia /El Salvador',:].index[0], "State"] = 'Georgia'

In [599]:
st123.State.value_counts().tail(15)

Delaware               3
Alaska                 3
Brazil                 2
Scotland               2
Maine                  1
Canada                 1
Haiti                  1
Montana                1
Jamaica                1
UK                     1
Vermont                1
U.S. Virgin Islands    1
South Africa           1
Philippines            1
Samoa                  1
Name: State, dtype: int64

In [600]:
st123.Season = st123.Season.apply(int)

In [601]:
st123["Country"] = np.where(st123["State"] == 'N/A', 'N/A',
                            np.where(st123["State"].str.strip().isin(states_list), "USA", "RoW"))

In [602]:
st123.head(30)

,Season,Singer,Age,Song,State,Country
0,16,Gyth Rigdon,24,"""Drift Away""",Louisiana,USA
1,16,Maelyn Jarmon,26,"""Fields of Gold""",Texas,USA
2,16,Karen Galera,19,"""Mi Corazoncito""",Texas,USA
3,16,Trey Rose,27,"""Wake Me Up""",Oklahoma,USA
4,16,Kim Cherry,30,"""No Scrubs""",Florida,USA
5,16,AJ Ryan,30,"""Love Runs Out""",New York,USA
6,16,Rizzi Myers,29,"""Breathin""",Tennessee,USA
7,16,Lisa Ramey,33,"""Sex on Fire""",New York,USA
8,16,Jimmy Mowery,31,"""Attention""",Pennsylvania,USA
9,16,LiLi Joy,15,"""Cool""",California,USA


In [615]:
st123.reset_index(inplace=True)

In [616]:
st123["Season_id"] = st123.groupby('Season')["index"].rank(method='first').apply(int)

In [617]:
column_order_2 = ["Season", "Season_id", "Singer", "Age", "Song", "State", "Country"]

In [618]:
st123 = st123[column_order_2]

In [619]:
st123.head()

,Season,Season_id,Singer,Age,Song,State,Country
0,16,1,Gyth Rigdon,24,"""Drift Away""",Louisiana,USA
1,16,2,Maelyn Jarmon,26,"""Fields of Gold""",Texas,USA
2,16,3,Karen Galera,19,"""Mi Corazoncito""",Texas,USA
3,16,4,Trey Rose,27,"""Wake Me Up""",Oklahoma,USA
4,16,5,Kim Cherry,30,"""No Scrubs""",Florida,USA


In [ ]:
source_df[[col, hinge_col]].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)

In [35]:
np.where(semi_table_2[['Song', 'Coach_1']].apply(lambda row: ', '.join(row.values.astype(str)), axis=1)

40                                     Cincinnati,  Ohio
70                          "Hold On,  We're Going Home"
30                              New York, "Walk Me Home"
67                      "Baby,  Now That I've Found You"
85                                 California, "Finesse"
47                                   "No Woman,  No Cry"
52                                    Texas, "Despacito"
54                               New York, "Lay Me Down"
23                  "Don't Think Twice,  It's All Right"
47                            New York, "Big White Room"
59                                   New York, "Imagine"
72                          "Hold On,  We're Going Home"
74                              New York, "Kozmic Blues"
81            "Heaven,  Heartache and the Power of Love"
56                        Canada / Nashville,  Tennessee
69                               England, "Viva la Vida"
73                          New York, "Don't Think Twice
74                       "Do Ri

In [ ]:

songs_list = [["Song A", "Song B"], ["Live Like You Were Dying", "versace on the floor"]]

In [ ]:
songs_list

In [ ]:
songs_cols = songs_list[0]
songs_rows = songs_list[0:2]

In [ ]:
songs = pd.DataFrame(songs_rows, columns=songs_cols)

In [ ]:
songs

In [ ]:
songs.iloc[1,:]

In [ ]:
songs.iloc[1,1]

In [ ]:
try: 
    from googlesearch import search 
except ImportError: 
    print("No module named 'google' found") 

# to search 
query = dataframe.iloc[1,5]

for j in search(query, num=5, stop=5, pause=5): 
    print(j) 


In [ ]:
url = "https://en.wikipedia.org/wiki/Live_Like_You_Were_Dying"
html = requests.get(url).content
soup = BeautifulSoup(html, "lxml")

In [ ]:
cosa = soup.find_all('table',{'class':'infobox vevent haudio'})[0]

In [ ]:
cosa

In [ ]:
cosa_row = cosa.find_all('tr')

In [ ]:
cosa_row

In [ ]:
vamos = [row.text for row in cosa_row]

In [ ]:
vamos

In [ ]:
cosa_row

In [ ]:
cosa.text

In [ ]:
cosota = cosa.text
lookup = 'Genre'
res = cosota.partition(lookup)[2][:16]

In [ ]:
res

In [ ]:
url = "https://en.wikipedia.org/wiki/The_Voice_(American_season_14)"
html = requests.get(url).content
soup = BeautifulSoup(html, "lxml")

In [ ]:
header = soup.find_all('span',{'class':'mw-headline', 'id':True})['id']

In [ ]:
header

In [ ]:
len(soup.find_all('h3'))

In [ ]:
heds = soup.find_all('h3')

In [ ]:
testeo = [row.text for row in heds]

In [ ]:
type(testeo)

In [ ]:
testeo = [row.text for row in soup.find_all('h3')]

In [ ]:
testeo

In [ ]:
sum(map(lambda x: "Episode" in x, testeo))